In [5]:
# Z-matrix形式のデータに対し、二面角の符号を反転させます。
import glob
import os
import json

In [27]:
data_dir = ""
paths = [p for p in glob.glob(data_dir)]

# result_dir = "data/output/" + "/".join(data_dir.split('/')[2:5]) + "/"

# ダミーを参照しているdについては操作しない→d5, d6, d7はそのままにする
# この部分を反転
# nd8   -179.3668\nd9   -0.5809\nd10   -0.5579\nd11   0.9613\nd12   176.4438\nd13   20.2222\nd14   -176.0902\nd15   -51.2261\nd16   -171.7808\nd17   -1.2794\nd18   7.4285\nd19   -2.5898\nd20   -1.1164\nd21   -0.1404\nd22   0.9022\nd23   -1.3428\nd24   3.8810\nd25   -10.5956\nd26   -6.9668\nd27   -2.6875\nd28   -2.9460\nd29   2.5538\nd30

start_key = "\nd8"
end_key = "\nConstants"

for path in paths:
    result_dir = path.replace('data/input/', 'data/output/')                 
    with open(path,"r") as f:
        data=f.read()
        # 文字列置換
        start = data.find(start_key)
        end = data.find(end_key)
        target_list = data[start:end]
        initial_d_list = []
        for dn in target_list.split('\n'):
            d_angle = dn.split(' ')[-1]
            initial_d_list.append(d_angle)
#         空の要素を削除
        initial_d_list = list(filter(None, initial_d_list))
# initial_d_listを使ってtarget_listを検索し、マッチした部分を-1倍する
        for initial_d in initial_d_list:
            data = data.replace(initial_d, str(float(initial_d) * -1))
        f.close()
        # 同じファイル名で保存
    with open(path, mode="w", encoding="cp932") as f:
        f.write(data)
        f.close()
#         同じファイル名で保存
#     with open(path, mode="w", encoding="cp932") as f:
#         f.write(path.replace('data/input/', 'data/output/'))

        
# 周辺分子を貼り付ける。多形ごとのデータを別ファイルで保管し、importして使う。


# 計算方法を書き換える


# ファイル名を編集

SyntaxError: invalid syntax (<ipython-input-27-9c1579458ae0>, line 31)

In [32]:
print(initial_d_list)
target_list

['179.3668', '0.5809', '0.5579', '-0.9613', '-176.4438', '-20.2222', '176.0902', '51.2261', '171.7808', '1.2794', '-7.4285', '2.5898', '1.1164', '0.1404', '-0.9022', '1.3428', '-3.881', '10.5956', '6.9668', '2.6875', '2.946', '-2.5538', '-8.9251']


'\nd8   179.3668\nd9   0.5809\nd10   0.5579\nd11   -0.9613\nd12   -176.4438\nd13   -20.2222\nd14   176.0902\nd15   51.2261\nd16   171.7808\nd17   1.2794\nd18   -7.4285\nd19   2.5898\nd20   1.1164\nd21   0.1404\nd22   -0.9022\nd23   1.3428\nd24   -3.881\nd25   10.5956\nd26   6.9668\nd27   2.6875\nd28   2.946\nd29   -2.5538\nd30   -8.9251'

In [12]:
import json
json_open = open('data/SA_data.json', 'r')
json_load = json.load(json_open)

print(json_load)

FileNotFoundError: [Errno 2] No such file or directory: 'data/SA_data.json'

In [33]:
data

'%chk=default\n%mem=2GB\n%NProcShared=6\n\n#N HF/3-21G Opt=Z-matrix Freq=(HPMode,Raman) \n\ncomments\n\n0 1\nX\nX   1   r2\nX   1   r3   2   a3\nX   1   r4   2   a4   3   d4\nH   1   r5   2   a5   3   d5\nC   5   r6   1   a6   2   d6\nC   6   r7   5   a7   1   d7\nC   7   r8   6   a8   5   d8\nC   8   r9   7   a9   6   d9\nC   9   r10   8   a10   7   d10\nC   10   r11   9   a11   8   d11\nN   9   r12   8   a12   7   d12\nC   12   r13   9   a13   8   d13\nC   13   r14   12   a14   9   d14\nC   14   r15   13   a15   12   d15\nC   15   r16   14   a16   13   d16\nC   16   r17   15   a17   14   d17\nC   17   r18   16   a18   15   d18\nC   18   r19   17   a19   16   d19\nH   11   r20   6   a20   5   d20\nH   10   r21   11   a21   20   d21\nH   7   r22   6   a22   5   d22\nH   8   r23   7   a23   22   d23\nH   12   r24   9   a24   10   d24\nH   13   r25   12   a25   9   d25\nH   19   r26   14   a26   13   d26\nH   18   r27   19   a27   26   d27\nH   17   r28   18   a28   27   d28\nH   16   r2